# Example of using the orm classes

Imports

In [1]:
from os import remove
from utils.io import striplog_from_text
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [2]:
remove('./tmp/test_orm_db.db')

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2021-01-19 01:15:18,386 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-19 01:15:18,388 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:18,395 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-19 01:15:18,397 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:18,402 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-01-19 01:15:18,406 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:18,415 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-01-19 01:15:18,417 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:18,420 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-01-19 01:15:18,422 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:18,427 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-01-19 01:15:18,432 INFO sqlalchemy.engine.base.Engine ()
2021-01-19

### Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':ROOT_DIR+'/data/test.las', 'F02':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
x = [0., 20.]
y = [0., 120.]

for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    #with open(filename, 'r') as las3:
    #    strip = Striplog.from_las3(las3.read(), lexicon)
    
    strip=striplog_from_text(filename)
    
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, middle=interval.top.middle,  
                          lower=interval.top.lower, x=x[bh_id], y=y[bh_id])
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, middle=interval.base.middle,  
                           lower=interval.base.lower, x=x[bh_id], y=y[bh_id])
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number, 
                          'top': top, 'base': base 
                         }})
        interval_number+=1
        int_id += 1
        pos_id += 2

 

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

File /home/q/GSDMA/data/test.las OK! Creation of the striplog ...
{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f17c9ef0860>, 'base': <core.orm.PositionOrm object at 0x7f17c9e8a278>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f17c9e8a198>, 'base': <core.orm.PositionOrm object at 0x7f17c9e8a470>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f17c9e8a6d8>, 'base': <core.orm.PositionOrm object at 0x7f17c9e8a208>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f17c9e8ad30>, 'base': <core.orm.PositionOrm object at 0x7f17c9e8a7f0>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f17c9e8a7b8>, 'base': <core.orm.PositionOrm object at 0x7f17c9e8a4e0>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 'top': <core.orm

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)
p.add_components(components)

2021-01-19 01:15:20,068 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-19 01:15:20,073 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-01-19 01:15:20,077 INFO sqlalchemy.engine.base.Engine ()
2021-01-19 01:15:20,084 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-01-19 01:15:20,088 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-01-19 01:15:20,092 INFO sqlalchemy.engine.base.Engine COMMIT
2021-01-19 01:15:20,274 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-19 01:15:20,284 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-01-19 01:15:20,288 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2021-01-19 01:15:20,335 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-01-19 01:15:20,339 INFO sqlalchemy.engine.base.Engine ('F01',)
2021-01-19 01:15:20,356 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-01-19 01:15:20,365 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.476470588

2021-01-19 01:15:20,665 INFO sqlalchemy.engine.base.Engine (14,)
2021-01-19 01:15:20,667 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:20,669 INFO sqlalchemy.engine.base.Engine (15,)
2021-01-19 01:15:20,673 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:20,676 INFO sqlalchemy.engine.base.Engine (16,)
2021-01-19 01:15:20,679 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-01-19 01:15:20,788 INFO sqlalchemy.engine.base.Engine (35,)
2021-01-19 01:15:20,791 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:20,793 INFO sqlalchemy.engine.base.Engine (36,)
2021-01-19 01:15:20,796 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:20,798 INFO sqlalchemy.engine.base.Engine (37,)
2021-01-19 01:15:20,801 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-01-19 01:15:21,372 INFO sqlalchemy.engine.base.Engine (52,)
2021-01-19 01:15:21,378 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:21,382 INFO sqlalchemy.engine.base.Engine (53,)
2021-01-19 01:15:21,390 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-01-19 01:15:21,394 INFO sqlalchemy.engine.base.Engine (54,)
2021-01-19 01:15:21,399 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.commit()

2021-01-19 01:15:21,779 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
p.boreholes[0].intervals[0].description

2021-01-19 01:15:21,969 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-19 01:15:21,973 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-01-19 01:15:21,975 INFO sqlalchemy.engine.base.Engine ('F01',)
2021-01-19 01:15:21,980 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-01-19 01:15:21,982 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [14]:
session.close()

2021-01-19 01:15:22,099 INFO sqlalchemy.engine.base.Engine ROLLBACK
